In [1]:
import pandas as pd

In [ ]:
df_00 = pd.read_csv('dataset/sensing/activity/activity_u00.csv')
df_00['date-time'] = pd.to_datetime(df_00['timestamp'],unit='s')
df_00['date'] = df_00['date-time'].dt.date
df_00

,timestamp,activity inference,date-time,date
0,1364356801,0,2013-03-27 04:00:01,2013-03-27
1,1364356804,0,2013-03-27 04:00:04,2013-03-27
2,1364356807,0,2013-03-27 04:00:07,2013-03-27
3,1364356809,0,2013-03-27 04:00:09,2013-03-27
4,1364356992,0,2013-03-27 04:03:12,2013-03-27
...,...,...,...,...
461986,1370048602,0,2013-06-01 01:03:22,2013-06-01
461987,1370048605,0,2013-06-01 01:03:25,2013-06-01
461988,1370048608,3,2013-06-01 01:03:28,2013-06-01
461989,1370048610,3,2013-06-01 01:03:30,2013-06-01


In [12]:
df_00.nunique()

timestamp              461751
 activity inference         4
date-time              461751
date                       66
dtype: int64

In [14]:
df_00 = df_00.sort_values(by='date-time')

In [19]:
df_00[' activity inference']

0         0
1         0
2         0
3         0
4         0
         ..
461986    0
461987    0
461988    3
461989    3
461990    3
Name:  activity inference, Length: 461991, dtype: int64

In [ ]:
# Taking the data only when there is a change in motion
motion_chang = df_00[df_00[' activity inference'] != df_00[' activity inference'].shift(1)]

In [ ]:
# Calculating the duration of that motion (if stationary, how long ? )
motion_chang.loc[:,'duration_in_s'] = motion_chang['date-time'].diff().dt.seconds.shift(-1)

In [53]:
motion_chang

,timestamp,activity inference,date-time,date,duration_in_s
0,1364356801,0,2013-03-27 04:00:01,2013-03-27,8163.0
773,1364364964,3,2013-03-27 06:16:04,2013-03-27,5.0
775,1364364969,0,2013-03-27 06:16:09,2013-03-27,216.0
789,1364365185,3,2013-03-27 06:19:45,2013-03-27,10.0
793,1364365195,0,2013-03-27 06:19:55,2013-03-27,13.0
...,...,...,...,...,...
461937,1370048090,2,2013-06-01 00:54:50,2013-06-01,236.0
461958,1370048326,0,2013-06-01 00:58:46,2013-06-01,31.0
461970,1370048357,3,2013-06-01 00:59:17,2013-06-01,16.0
461976,1370048373,0,2013-06-01 00:59:33,2013-06-01,235.0


In [ ]:
# Taking the duration only when the person is moving (walking / running)  # label 0 = stationary & 3 = unknown
motion_chang.loc[:,'motion_duration_s'] = motion_chang['duration_in_s'] * (motion_chang[' activity inference'].isin([1,2]).astype('int'))

In [70]:
motion_chang

,timestamp,activity inference,date-time,date,duration_in_s,motion_duration_s
0,1364356801,0,2013-03-27 04:00:01,2013-03-27,8163.0,0.0
773,1364364964,3,2013-03-27 06:16:04,2013-03-27,5.0,0.0
775,1364364969,0,2013-03-27 06:16:09,2013-03-27,216.0,0.0
789,1364365185,3,2013-03-27 06:19:45,2013-03-27,10.0,0.0
793,1364365195,0,2013-03-27 06:19:55,2013-03-27,13.0,0.0
...,...,...,...,...,...,...
461937,1370048090,2,2013-06-01 00:54:50,2013-06-01,236.0,236.0
461958,1370048326,0,2013-06-01 00:58:46,2013-06-01,31.0,0.0
461970,1370048357,3,2013-06-01 00:59:17,2013-06-01,16.0,0.0
461976,1370048373,0,2013-06-01 00:59:33,2013-06-01,235.0,0.0


In [ ]:
final_df = motion_chang.groupby('date').agg(
    total_duration_of_motion_sensing = ('motion_duration_s','sum'),
)

In [ ]:
def get_sensing_activity_data(filepath):
    df_00 = pd.read_csv(filepath)
    df_00['date-time'] = pd.to_datetime(df_00['timestamp'],unit='s')
    df_00['date'] = df_00['date-time'].dt.date
    df_00 = df_00.sort_values(by='date-time')
    filtered_df = df_00[df_00[' activity inference'] != df_00[' activity inference'].shift(1)].copy()
    filtered_df['duration_in_s'] = filtered_df['date-time'].diff().dt.total_seconds().shift(-1)
    filtered_df['duration_in_s'] = filtered_df['duration_in_s'].fillna(0)
    filtered_df['motion_duration_s'] = filtered_df['duration_in_s'] * (filtered_df[' activity inference'].isin([1,2]).astype('int'))

    final_df = filtered_df.groupby('date').agg(
        total_duration_of_motion_sensing = ('motion_duration_s','sum'),
    )
    return final_df

In [13]:
get_sensing_activity_data('dataset/sensing/activity/activity_u00.csv')

,total_duration_of_motion_sensing
date,
2013-03-27,11984.0
2013-03-28,6437.0
2013-03-29,12335.0
2013-03-30,4219.0
2013-03-31,14490.0
...,...
2013-05-28,7479.0
2013-05-29,11525.0
2013-05-30,6721.0


In [14]:
df_00 = pd.read_csv('dataset/sensing/conversation/conversation_u00.csv')
df_00.columns = df_00.columns.str.strip()
df_00['start_timestamp'] = pd.to_datetime(df_00['start_timestamp'],unit='s')
df_00['end_timestamp'] = pd.to_datetime(df_00['end_timestamp'],unit='s')
df_00['date'] = df_00['start_timestamp'].dt.date
df_00

,start_timestamp,end_timestamp,date
0,2013-03-27 04:46:40,2013-03-27 04:50:12,2013-03-27
1,2013-03-27 11:10:21,2013-03-27 11:17:45,2013-03-27
2,2013-03-27 11:25:16,2013-03-27 11:49:53,2013-03-27
3,2013-03-27 11:50:33,2013-03-27 11:51:34,2013-03-27
4,2013-03-27 12:03:06,2013-03-27 12:04:26,2013-03-27
...,...,...,...
2106,2013-05-31 21:57:23,2013-05-31 22:00:04,2013-05-31
2107,2013-05-31 22:04:34,2013-05-31 22:18:30,2013-05-31
2108,2013-05-31 22:22:11,2013-05-31 23:54:18,2013-05-31
2109,2013-05-31 23:58:08,2013-06-01 00:53:46,2013-05-31


In [ ]:
df_00['conversation_duration'] = (df_00['end_timestamp'] - df_00['start_timestamp']).dt.total_seconds()
df_00

,start_timestamp,end_timestamp,date,conversation_duration
0,2013-03-27 04:46:40,2013-03-27 04:50:12,2013-03-27,212.0
1,2013-03-27 11:10:21,2013-03-27 11:17:45,2013-03-27,444.0
2,2013-03-27 11:25:16,2013-03-27 11:49:53,2013-03-27,1477.0
3,2013-03-27 11:50:33,2013-03-27 11:51:34,2013-03-27,61.0
4,2013-03-27 12:03:06,2013-03-27 12:04:26,2013-03-27,80.0
...,...,...,...,...
2106,2013-05-31 21:57:23,2013-05-31 22:00:04,2013-05-31,161.0
2107,2013-05-31 22:04:34,2013-05-31 22:18:30,2013-05-31,836.0
2108,2013-05-31 22:22:11,2013-05-31 23:54:18,2013-05-31,5527.0
2109,2013-05-31 23:58:08,2013-06-01 00:53:46,2013-05-31,3338.0


In [ ]:
final_df = df_00.groupby('date').agg(
    total_daily_conversation = ('conversation_duration','sum')
)


,total_daily_conversation
date,
2013-03-27,22971.0
2013-03-28,20902.0
2013-03-29,28747.0
2013-03-30,31682.0
2013-03-31,35861.0
...,...
2013-05-28,32764.0
2013-05-29,30256.0
2013-05-30,19486.0


In [22]:
# Get Sensing-conversation
def get_sensing_conversation_df(filepath):
    df_00 = pd.read_csv(filepath)
    df_00.columns = df_00.columns.str.strip()
    df_00['start_timestamp'] = pd.to_datetime(df_00['start_timestamp'],unit='s')
    df_00['end_timestamp'] = pd.to_datetime(df_00['end_timestamp'],unit='s')
    df_00['date'] = df_00['start_timestamp'].dt.date
    df_00['conversation_duration'] = (df_00['end_timestamp'] - df_00['start_timestamp']).dt.total_seconds()
    final_df = df_00.groupby('date').agg(
        total_daily_conversation = ('conversation_duration','sum')
    )
    return final_df

In [2]:
def get_sensing_activity_data(filepath):
    df_00 = pd.read_csv(filepath)
    df_00['date-time'] = pd.to_datetime(df_00['timestamp'],unit='s')
    df_00['date'] = df_00['date-time'].dt.date
    df_00 = df_00.sort_values(by='date-time')
    filtered_df = df_00[df_00[' activity inference'] != df_00[' activity inference'].shift(1)].copy()
    filtered_df['duration_in_s'] = filtered_df['date-time'].diff().dt.total_seconds().shift(-1)
    filtered_df['duration_in_s'] = filtered_df['duration_in_s'].fillna(0)
    filtered_df['motion_duration_s'] = filtered_df['duration_in_s'] * (filtered_df[' activity inference'].isin([1,2]).astype('int'))

    final_df = filtered_df.groupby('date').agg(
        total_duration_of_motion_sensing = ('motion_duration_s','sum'),
    )
    return final_df

def get_sensing_conversation_df(filepath):
    df_00 = pd.read_csv(filepath)
    df_00.columns = df_00.columns.str.strip()
    df_00['start_timestamp'] = pd.to_datetime(df_00['start_timestamp'],unit='s')
    df_00['end_timestamp'] = pd.to_datetime(df_00['end_timestamp'],unit='s')
    df_00['date'] = df_00['start_timestamp'].dt.date
    df_00['conversation_duration'] = (df_00['end_timestamp'] - df_00['start_timestamp']).dt.total_seconds()
    final_df = df_00.groupby('date').agg(
        total_daily_conversation = ('conversation_duration','sum')
    )
    return final_df

In [ ]:

import os
user = '01'
def get_student_df(root_dir, student_number:str):
    # activity = get_sensing_activity_data(f'dataset/sensing/activity/activity_u{student_number}.csv')
    activity = get_sensing_activity_data(os.path.join(root_dir,f'activity/activity_{student_number}.csv'))
    conversation = get_sensing_conversation_df(os.path.join(root_dir,f'conversation/conversation_{student_number}.csv'))
    student_df = pd.concat([activity,conversation],axis=1)
    student_df['uid'] = student_number
    return student_df

In [10]:
# get_student_df('00')

import os

root_dir = '../dataset/sensing'
exercise_folder = 'activity'
# os.path.join('dataset','app_usage')
user_files = os.listdir(path=os.path.join(root_dir,exercise_folder))

uids = [x.split('_')[-1].replace('.csv','') for x in user_files]

combined_dfs = []
for uid in uids:
    print('Ingesting User : ',uid)
    
    df= get_student_df(root_dir=root_dir,student_number=uid)
    combined_dfs.append(df)
ema_df = pd.concat(combined_dfs)

Ingesting User :  u49
Ingesting User :  u51
Ingesting User :  u57
Ingesting User :  u56
Ingesting User :  u13
Ingesting User :  u52
Ingesting User :  u30
Ingesting User :  u10
Ingesting User :  u08
Ingesting User :  u05
Ingesting User :  u36
Ingesting User :  u00
Ingesting User :  u17
Ingesting User :  u50
Ingesting User :  u34
Ingesting User :  u20
Ingesting User :  u15
Ingesting User :  u02
Ingesting User :  u24
Ingesting User :  u09
Ingesting User :  u16
Ingesting User :  u23
Ingesting User :  u07
Ingesting User :  u45
Ingesting User :  u19
Ingesting User :  u53
Ingesting User :  u31
Ingesting User :  u44
Ingesting User :  u39
Ingesting User :  u03
Ingesting User :  u59
Ingesting User :  u04
Ingesting User :  u54
Ingesting User :  u18
Ingesting User :  u43
Ingesting User :  u41
Ingesting User :  u47
Ingesting User :  u58
Ingesting User :  u14
Ingesting User :  u25
Ingesting User :  u33
Ingesting User :  u22
Ingesting User :  u27
Ingesting User :  u42
Ingesting User :  u32
Ingesting 

In [11]:
ema_df

,total_duration_of_motion_sensing,total_daily_conversation
date,,
2013-03-27,0.0,NaN
2013-03-28,1204.0,4438.0
2013-03-29,6725.0,18527.0
2013-03-30,3182.0,14398.0
2013-03-31,5526.0,17669.0
...,...,...
2013-05-19,2683.0,16598.0
2013-05-20,12202.0,23458.0
2013-05-21,23510.0,43508.0
